# Natural Language Processing

P.S. to run again the code, please use the .csv files existed in this .zip.
The data columns had to be named. so, a slight change in .txt files load data into pandas dataframes.

## Importing the libraries

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
from sklearn.metrics import confusion_matrix, accuracy_score

# Importing the dataset

In [2]:
dataset = pd.read_csv('r8-train-all-terms.txt', delimiter='\t', quoting=3)
test_dataset = pd.read_csv('r8-train-all-terms.txt',delimiter='\t', quoting=3)
dataset = dataset[pd.notnull(dataset['Example'])]
test_dataset = test_dataset[pd.notnull(dataset['Example'])]

test_dataset

,Class,Example
0,earn,champion products ch approves stock split cham...
1,acq,computer terminal systems cpml completes sale ...
2,earn,cobanco inc cbco year net shr cts vs dlrs net ...
3,earn,am international inc am nd qtr jan oper shr lo...
4,earn,brown forman inc bfd th qtr net shr one dlr vs...
...,...,...
5480,earn,kelly oil and gas partners kly year dec shr ct...
5481,money-fx,japan seeks to strengthen paris currency accor...
5482,earn,tcw convertible securities cvt sets dividend t...
5483,money-fx,south korean won fixed at month high the bank ...


# Cleaning the texts

In [3]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
replace_by_space_re = re.compile('[/(){}\[\]\|@,;]')
bad_symbols_re = re.compile('[^0-9a-z #+_]')